In [1]:
from src.llm.generator import LLamaCppGeneratorComponent

In [2]:
prompt = "You are a french news reporter"

In [3]:
%load_ext dotenv

In [4]:
%dotenv ./.env_local -o

In [5]:
llama_cpp_generator = LLamaCppGeneratorComponent(
    api_url="http://127.0.0.1:8001",
    prompt=prompt
)

In [6]:
llama_cpp_generator._ping_api()

True

### Download the File

In [10]:
filename = "news-clusters-2024-11-20-to-2024-11-18.csv"

In [11]:
BUCKET_NAME = "congonews-clusters"

In [13]:
from tempfile import NamedTemporaryFile

In [30]:
from csv import DictReader as csv_reader

In [ ]:
from itertools import groupby

In [52]:
from unicodedata import normalize

In [18]:
today_news_df.sort_values(by="labels")

,database_id,content,title,posted_at,url,labels
92,31562427,En sa qualité de coordonnateur du Conseil prés...,Sud global/Chine : François Muamba en appelle ...,2024-11-20 00:00:00,https://www.mediacongo.net/article-actualite-1...,4
138,31636465,Au cours de son intervention aux assises lors ...,Global South Think Tanks : François Muamba Tsh...,2024-11-20 11:09:00,https://actualite.cd/index.php/2024/11/20/glob...,4
140,31000142,La Première Ministre de la République Démocrat...,Conférence Africa Accelerating 2024 : Judith S...,2024-11-18 00:00:00,https://www.mediacongo.net/article-actualite-1...,6
111,31562749,Le Gouvernement attend d'abord du Canada d'êtr...,Judith Suminwa appelle le Canada à soutenir le...,2024-11-20 00:00:00,https://www.mediacongo.net/article-actualite-1...,6
17,31345599,"novembre 18, 2024 RDC : “Je suis là pour obten...",RDC : “Je suis là pour obtenir des résultats” ...,2024-11-18 00:00:00,https://depeche.cd/2024/11/18/rdc-je-suis-la-p...,6
...,...,...,...,...,...,...
82,31241267,La Cour de cassation siégeant en matière répre...,Dossier Forages: la Cour a pris en délibéré la...,2024-11-19 07:59:00,https://actualite.cd/index.php/2024/11/19/doss...,89
203,31288342,Après le moyen exceptionnel soulevé par la déf...,Instruction du procès « forages » : Mike Kasen...,2024-11-19 06:32:16,https://scooprdc.net/2024/11/19/instruction-du...,89
133,31261209,L’instruction du dossier des constructions des...,"Selon François Rubota, l’idée de construire 1 ...",2024-11-19 09:59:00,https://actualite.cd/2024/11/19/selon-francois...,89
32,30996093,L’affaire mettant en cause François Rubota et ...,Dossier forages : les avocats de François Rubo...,2024-11-18 12:56:00,https://actualite.cd/2024/11/18/dossier-forage...,89


### Prompt

In [19]:
from pprint import pprint

In [57]:
from itertools import groupby

In [59]:
summaries = []
def sort_function(x): return x["labels"]


sorted_data = sorted(data, key=sort_function)
grouped_data = groupby(sorted_data, key=sort_function)
for label, group in grouped_data:
    news_data = list(group)
    titles = [news["title"] for news in news_data]
    urls = [news["url"] for news in news_data]
    content = "\n".join([news["content"] for news in news_data])
    content = normalize("NFKD", content)
    summary = llama_cpp_generator.run(
        template_values={"content": content}, prompt_template=summarization_prompt)
    news_data = {
        "titles": titles,
        "urls": urls,
        "summary": summary
    }
    summaries.append(news_data)
    print(f" This is the summary for label {label}")
    pprint(summary)
    print(f"Here are the titles for label {label}")
    print(10 * "*")
    for title in titles:
        print(title)
    print(10 * "*")

 This is the summary for label 10
('La Belgique condamne le soutien du Rwanda au M23 et appelle le pays à '
 "retirer ses troupes de l'Est de la RDC. Le Conseil de l'Union européenne a "
 "adopté une enveloppe supplémentaire de 20 millions d'euros pour renforcer "
 'les RDF dans la province de Cabo Delgado, au Mozambique, en soutien à la '
 'lutte contre le terrorisme. Cette aide est critiquée par Kinshasa et la '
 "société civile congolaise, qui s'inquiètent de son implication dans le "
 'conflit au Nord-Kivu. La Belgique a également renouvelé son soutien aux '
 'troupes rwandaises engagées dans la lutte contre le terrorisme au '
 'Mozambique, malgré les critiques de Kinshasa.')
Here are the titles for label 10
**********
Guerre de l’Est : la Belgique condamne le soutien du Rwanda au M23
L'UE renforce son soutien aux troupes rwandaises au Mozambique, malgré les critiques récurrentes de Kinshasa | Actualite.cd
Malgré les pressions de la RDC : l’UE renouvelle son aide financière aux tro

I will keep both model in the server and use them. I will be benchmackring them.

Chat template without huggingface.

In [25]:
import json

In [28]:
with open("news_summaries.json", "w") as f:
    json.dump(summaries, f, ensure_ascii=False)

In [61]:
summaries
for summary in summaries:
    pprint(summary["summary"])
    for title_ in summary["titles"]:
        print(title_)
    print(10 * "*")

    print(10 * "*")

('La Belgique condamne le soutien du Rwanda au M23 et appelle le pays à '
 "retirer ses troupes de l'Est de la RDC. Le Conseil de l'Union européenne a "
 "adopté une enveloppe supplémentaire de 20 millions d'euros pour renforcer "
 'les RDF dans la province de Cabo Delgado, au Mozambique, en soutien à la '
 'lutte contre le terrorisme. Cette aide est critiquée par Kinshasa et la '
 "société civile congolaise, qui s'inquiètent de son implication dans le "
 'conflit au Nord-Kivu. La Belgique a également renouvelé son soutien aux '
 'troupes rwandaises engagées dans la lutte contre le terrorisme au '
 'Mozambique, malgré les critiques de Kinshasa.')
Guerre de l’Est : la Belgique condamne le soutien du Rwanda au M23
L'UE renforce son soutien aux troupes rwandaises au Mozambique, malgré les critiques récurrentes de Kinshasa | Actualite.cd
Malgré les pressions de la RDC : l’UE renouvelle son aide financière aux troupes rwandaises au Mozambique
**********
**********
('La Société Congolaise de